# **REGEX and Data Processing**

In this notebook, we explore various datasets using professional data processing techniques to perform comprehensive data analysis. We will clean data, merge datasets, parse logs, and conduct exploratory data analysis on restaurant violation data.

***

## **1. Import Necessary Libraries**

Start by importing the required Python libraries

In [1]:
import re
import pandas as pd

## **2. Loading and Displaying Data**

Load the datasets and display the first few rows to understand their structure

In [2]:
# Load datasets
county_and_state = pd.read_csv('../data/county_and_state.csv')
county_and_pop = pd.read_csv('../data/county_and_population.csv')

# Display the first few rows of the first dataset
display(county_and_state.head())

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


In [3]:
# Display the first few rows of the second dataset
display(county_and_pop.head())

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


## **3. Data Cleaning**

Define a function to clean county names for consistent formatting and apply this function to the datasets

In [4]:
def clean_county(county):
    """
    Cleans the county name by converting to lowercase and removing spaces, special characters, and certain words.
    """
    return county.lower().replace(" ", "").replace("&", "").replace("and", "").replace("county", "").replace("parish", "").replace(".", "")

county_and_state["cleaned_county"] = county_and_state["County"].map(clean_county)
county_and_pop["cleaned_county"] = county_and_pop["County"].map(clean_county)

# Display updated dataset
display(county_and_state)

,County,State,cleaned_county
0,De Witt County,IL,dewitt
1,Lac qui Parle County,MN,lacquiparle
2,Lewis and Clark County,MT,lewisclark
3,St John the Baptist Parish,LS,stjohnthebaptist


In [5]:
# Display updated dataset
display(county_and_pop)

,County,Population,cleaned_county
0,DeWitt,16798,dewitt
1,Lac Qui Parle,8067,lacquiparle
2,Lewis & Clark,55716,lewisclark
3,St. John the Baptist,43044,stjohnthebaptist


## **4. Merging Datasets**

Utilize the cleaned county names to merge the two datasets

In [6]:
merged_data = pd.merge(county_and_pop, county_and_state, on="cleaned_county")
display(merged_data)

,County_x,Population,cleaned_county,County_y,State
0,DeWitt,16798,dewitt,De Witt County,IL
1,Lac Qui Parle,8067,lacquiparle,Lac qui Parle County,MN
2,Lewis & Clark,55716,lewisclark,Lewis and Clark County,MT
3,St. John the Baptist,43044,stjohnthebaptist,St John the Baptist Parish,LS


## **5. Log File Parsing**

Read a log file and explore different methods to extract date and time information

In [7]:
# Read log file
with open('../data/log.txt', 'r') as file:
    log_lines = file.readlines()

# Display log lines
display(log_lines)

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

In [8]:
# Extract date and time using regular expressions
pattern = r'\[(\d+)/(\w+)/(\d+):(\d+):(\d+):(\d+) (.+)\]'
dates_times = [re.findall(pattern, line) for line in log_lines]
display(dates_times)

[[('26', 'Jan', '2014', '10', '47', '58', '-0800')],
 [('2', 'Feb', '2005', '17', '23', '6', '-0800')],
 [('3', 'Feb', '2006', '10', '18', '37', '-0800')]]

## **6. Exploratory Data Analysis**

Load the restaurant violation data and perform initial exploration

In [9]:
# Load restaurant violations data
vio = pd.read_csv("../data/violations.csv")
display(vio.head(10))

,business_id,date,description
0,19,20171211,Inadequate food safety knowledge or lack of ce...
1,19,20171211,Unapproved or unmaintained equipment or utensils
2,19,20160513,Unapproved or unmaintained equipment or utensi...
3,19,20160513,Unclean or degraded floors walls or ceilings ...
4,19,20160513,Food safety certificate or food handler card n...
5,24,20171101,Improper food storage
6,24,20161005,Unclean or degraded floors walls or ceilings ...
7,24,20160311,Unclean or degraded floors walls or ceilings ...
8,24,20160311,Unclean or degraded floors walls or ceilings ...
9,31,20151204,Food safety certificate or food handler card n...


In [10]:
# Analyzing the frequency of unique descriptions
# This helps understand common types of violations reported
unique_desc = vio['description'].value_counts()
print("Number of unique descriptions:", unique_desc.shape[0])

Number of unique descriptions: 14253


In [11]:
# Show a sample of descriptions from the middle of the frequency distribution
display(unique_desc[50:60])

description
Unclean or degraded floors walls or ceilings  [ date violation corrected: 11/29/2017 ]              16
Unclean or degraded floors walls or ceilings  [ date violation corrected: 9/19/2017 ]               16
Inadequate HACCP plan record keeping                                                                16
Unclean or degraded floors walls or ceilings  [ date violation corrected: 11/27/2017 ]              15
Unclean or degraded floors walls or ceilings  [ date violation corrected: 12/7/2017 ]               15
Inadequately cleaned or sanitized food contact surfaces  [ date violation corrected: 9/26/2017 ]    14
Unclean or degraded floors walls or ceilings  [ date violation corrected: 11/28/2017 ]              14
Unclean or degraded floors walls or ceilings  [ date violation corrected: 9/6/2017 ]                14
Unapproved or unmaintained equipment or utensils  [ date violation corrected: 9/19/2017 ]           14
Unapproved  living quarters in food facility                 

In [12]:
# Clean descriptions by removing additional information within brackets
# This standardizes the descriptions for better analysis
pattern = r"\[.+\]"  # Regex pattern to find any characters within brackets
vio["cleaned_description"] = vio["description"].str.replace(pattern, "", regex=True)
print("Number of unique cleaned descriptions:", vio["cleaned_description"].value_counts().shape[0])
display(vio[["description", "cleaned_description"]][50:60])

Number of unique cleaned descriptions: 129


,description,cleaned_description
50,Improper or defective plumbing [ date violati...,Improper or defective plumbing
51,High risk food holding temperature [ date vi...,High risk food holding temperature
52,High risk food holding temperature [ date vi...,High risk food holding temperature
53,Inadequate ventilation or lighting [ date vio...,Inadequate ventilation or lighting
54,Unclean or unsanitary food contact surfaces [...,Unclean or unsanitary food contact surfaces
55,Improper cooling methods [ date violation cor...,Improper cooling methods
56,Low risk vermin infestation [ date violation ...,Low risk vermin infestation
57,No thermometers or uncalibrated thermometers ...,No thermometers or uncalibrated thermometers
58,Foods not protected from contamination [ date...,Foods not protected from contamination
59,Unapproved or unmaintained equipment or utensils,Unapproved or unmaintained equipment or utensils


In [13]:
# Adding boolean columns for specific violation issues based on keywords in descriptions
# These columns will help in filtering and detailed analysis later on
features = ['high risk', 'vermin', 'wall|ceiling|floor|surface', 'hand|glove|hair|nail', 'permit|certif']
column_names = ['is_high_risk', 'is_vermin', 'is_surface', 'is_human', 'is_permit']

for feature, col_name in zip(features, column_names):
    vio[col_name] = vio['cleaned_description'].str.contains(feature)
display(vio.head(10))

,business_id,date,description,cleaned_description,is_high_risk,is_vermin,is_surface,is_human,is_permit
0,19,20171211,Inadequate food safety knowledge or lack of ce...,Inadequate food safety knowledge or lack of ce...,False,False,False,False,True
1,19,20171211,Unapproved or unmaintained equipment or utensils,Unapproved or unmaintained equipment or utensils,False,False,False,False,False
2,19,20160513,Unapproved or unmaintained equipment or utensi...,Unapproved or unmaintained equipment or utensi...,False,False,False,False,False
3,19,20160513,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False
4,19,20160513,Food safety certificate or food handler card n...,Food safety certificate or food handler card n...,False,False,False,True,True
5,24,20171101,Improper food storage,Improper food storage,False,False,False,False,False
6,24,20161005,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False
7,24,20160311,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False
8,24,20160311,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False
9,31,20151204,Food safety certificate or food handler card n...,Food safety certificate or food handler card n...,False,False,False,True,True


In [14]:
# Counting the number of violations per inspection
# This will give an insight into the compliance level of businesses over time
violation_counts = vio.groupby(['business_id', 'date']).size().reset_index(name='violation_count')
display(violation_counts.head(10))

,business_id,date,violation_count
0,19,20160513,3
1,19,20171211,2
2,24,20160311,2
3,24,20161005,1
4,24,20171101,1
5,31,20151204,1
6,45,20160104,7
7,45,20160614,5
8,45,20170307,5
9,45,20170914,4


In [15]:
# Analyzing the frequency of vermin-related violations
vermin_violations = vio[vio['is_vermin'] == True]
print("Number of vermin violations:", len(vermin_violations))

Number of vermin violations: 3386


In [16]:
# Determine the resolution status of violations based on whether a correction date is mentioned
vio['status'] = vio['description'].apply(lambda x: 'resolved' if 'date violation corrected' in x else 'unresolved')
display(vio.head(10))

,business_id,date,description,cleaned_description,is_high_risk,is_vermin,is_surface,is_human,is_permit,status
0,19,20171211,Inadequate food safety knowledge or lack of ce...,Inadequate food safety knowledge or lack of ce...,False,False,False,False,True,unresolved
1,19,20171211,Unapproved or unmaintained equipment or utensils,Unapproved or unmaintained equipment or utensils,False,False,False,False,False,unresolved
2,19,20160513,Unapproved or unmaintained equipment or utensi...,Unapproved or unmaintained equipment or utensi...,False,False,False,False,False,resolved
3,19,20160513,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False,resolved
4,19,20160513,Food safety certificate or food handler card n...,Food safety certificate or food handler card n...,False,False,False,True,True,resolved
5,24,20171101,Improper food storage,Improper food storage,False,False,False,False,False,unresolved
6,24,20161005,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False,resolved
7,24,20160311,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False,resolved
8,24,20160311,Unclean or degraded floors walls or ceilings ...,Unclean or degraded floors walls or ceilings,False,False,True,False,False,resolved
9,31,20151204,Food safety certificate or food handler card n...,Food safety certificate or food handler card n...,False,False,False,True,True,unresolved
